In [1]:
import datetime
import matplotlib.pyplot as plt
from matplotlib import pyplot
import pandas as pd
import sklearn
from sklearn.inspection import PartialDependenceDisplay
# from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
# from sklearn.tree import DecisionTreeRegressor
import numpy as np

pd.set_option("display.max_colwidth", 0)
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)
import warnings
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.5f}'.format

In [2]:
import sys
import logging
import datetime

nblog = open("./output/tree_pdp_shap_"+str(datetime.datetime.now())+".log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [3]:
%%time

df_traffic = pd.read_csv('./output/austin_2022_GP_10min_interval_delaydifference_with_features_forML.csv')
df_traffic['minutes_since_midnight'] = df_traffic['hour_min'].apply(lambda x: int(x[:2]) * 60 + int(x[3:]))

print(df_traffic.date.unique())
print('unique road segments (samples):',df_traffic.tmc_code.unique().shape[0])
print('total observations: ',df_traffic.shape[0])
df_traffic.head(2)

['2022-10-21' '2022-10-22' '2022-10-23']
unique road segments (samples): 4450
total observations:  1917950
CPU times: user 4.87 s, sys: 571 ms, total: 5.44 s
Wall time: 6.15 s


,tmc_code,hour_min,delay_baseline,delay_focus,delay_difference,date,intersection,start_latitude,start_longitude,end_latitude,end_longitude,miles,airbnb_count,distance_to_Shuttle_Waterloo_Park,distance_to_Shuttle_Barton_Creek_Square,distance_to_Shuttle_Expo_Center,distance_to_Uber_DelValle_HighSchool,distance_to_venue,minutes_since_midnight
0,112+04758,00:00,-0.93268,-0.93268,0.00000,2022-10-21,51ST ST/CAMERON RD/EXIT 237,30.30330,-97.71418,30.30530,-97.71289,0.15814,87,4685.01085,11838.72611,10171.97090,19603.37763,23552.59734,0
1,112+04758,00:10,-0.93268,-0.93268,0.00000,2022-10-21,51ST ST/CAMERON RD/EXIT 237,30.30330,-97.71418,30.30530,-97.71289,0.15814,87,4685.01085,11838.72611,10171.97090,19603.37763,23552.59734,10


In [4]:
# because the event usually begins in the morning after 10, people arrive typically around that time, so we want to only focus on after 930
df_traffic = df_traffic[df_traffic['minutes_since_midnight']>=540]
print(df_traffic.shape[0])
df_traffic = pd.get_dummies(df_traffic, columns=['date'], prefix='date')

1197050


In [5]:
df_traffic.columns

Index(['tmc_code', 'hour_min', 'delay_baseline', 'delay_focus', 'delay_difference', 'intersection', 'start_latitude', 'start_longitude', 'end_latitude', 'end_longitude', 'miles', 'airbnb_count', 'distance_to_Shuttle_Waterloo_Park', 'distance_to_Shuttle_Barton_Creek_Square', 'distance_to_Shuttle_Expo_Center', 'distance_to_Uber_DelValle_HighSchool', 'distance_to_venue', 'minutes_since_midnight', 'date_2022-10-21', 'date_2022-10-22', 'date_2022-10-23'], dtype='object')

In [6]:
# df_traffic_oneday = df_traffic[df_traffic['date']=='2022-10-22']

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Prepare input features and labels for the model
X = df_traffic[['minutes_since_midnight',  'start_latitude', 'start_longitude', 'miles',
                'date_2022-10-21', 'date_2022-10-22', 'date_2022-10-23',
                'airbnb_count', 'distance_to_Shuttle_Waterloo_Park', 'distance_to_Shuttle_Barton_Creek_Square', 
                'distance_to_Shuttle_Expo_Center', 'distance_to_Uber_DelValle_HighSchool', 'distance_to_venue']].copy()
y = df_traffic['delay_difference'].values

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('Total sample size:', X.shape[0])
print('Total training set size:', X_train.shape[0])
print('Total test set size:', X_test.shape[0])

Total sample size: 1197050
Total training set size: 837935
Total test set size: 359115


In [8]:
%%time
# check VIF
df_cor = X.corr()
pd.DataFrame(np.linalg.inv(X.corr().values), index = df_cor.index, columns=df_cor.columns)

CPU times: user 795 ms, sys: 48.6 ms, total: 843 ms
Wall time: 869 ms


,minutes_since_midnight,start_latitude,start_longitude,miles,date_2022-10-21,date_2022-10-22,date_2022-10-23,airbnb_count,distance_to_Shuttle_Waterloo_Park,distance_to_Shuttle_Barton_Creek_Square,distance_to_Shuttle_Expo_Center,distance_to_Uber_DelValle_HighSchool,distance_to_venue
minutes_since_midnight,1.00002,0.00000,0.00000,0.00000,-0.00707,-0.00706,-0.01203,0.00000,0.00000,-0.00000,-0.00000,0.00000,-0.00000
start_latitude,0.00000,16.66550,-0.25938,0.81057,-0.08225,-0.08225,-0.08202,0.23729,2.53443,-0.58231,0.53332,32.09939,-47.84499
start_longitude,0.00000,-0.25938,5.33612,0.29394,0.02715,0.02715,0.02708,-0.38134,-0.90740,-5.82432,-0.41746,18.09198,-13.01078
miles,0.00000,0.81057,0.29394,1.25955,0.10168,0.10168,0.10139,-0.14218,-1.03468,-0.02651,-0.00539,2.77891,-2.86867
date_2022-10-21,-0.00706,-0.08225,0.02715,0.10168,-22866999849187.30078,-22866999849188.27734,-22803036087951.56250,0.04071,-0.00819,-0.04723,0.05936,-0.24511,0.30592
date_2022-10-22,-0.00706,-0.08225,0.02715,0.10168,-22866999849188.27734,-22866999849187.92578,-22803036087951.87500,0.04071,-0.00819,-0.04723,0.05936,-0.24511,0.30592
date_2022-10-23,-0.01202,-0.08202,0.02708,0.10139,-22803036087951.56250,-22803036087951.87500,-22739251246676.81641,0.04060,-0.00816,-0.04710,0.05920,-0.24442,0.30507
airbnb_count,0.00000,0.23729,-0.38134,-0.14218,0.04071,0.04071,0.04060,1.70781,5.98059,-3.08213,-2.44925,1.00674,-1.28450
distance_to_Shuttle_Waterloo_Park,0.00000,2.53443,-0.90740,-1.03468,-0.00819,-0.00819,-0.00816,5.98059,74.45623,-50.27565,-22.59948,-18.79238,13.87097
distance_to_Shuttle_Barton_Creek_Square,-0.00000,-0.58231,-5.82432,-0.02651,-0.04723,-0.04723,-0.04710,-3.08213,-50.27565,46.38830,10.13752,15.60473,-18.15437


# Hyperparameter tuning using optuna

In [ ]:
# %%time
# # %pip install optuna
# ## this step took 10 hours to finish

# import optuna
# from sklearn.model_selection import cross_val_score, KFold
# from sklearn.metrics import make_scorer, mean_squared_error

# def objective(trial):
#     # Define hyperparameters to optimize
#     params = {
#         'iterations': trial.suggest_int('iterations', 100, 1200, step=100),
#         'max_depth': trial.suggest_int('depth', 4, 12),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.3),
#         "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
#         "subsample": trial.suggest_float("subsample", 0.05, 1.0),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
#         'loss_function': 'RMSE','eval_metric': 'RMSE'
#     }

#     # Initialize CatBoostRegressor with the suggested hyperparameters
#     model = cb.CatBoostRegressor(cat_features=categorical_list, **params, verbose=0)
    
#     # Perform cross-validation
#     kf = KFold(n_splits=5, shuffle=True, random_state=42)
# #     kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
#     rmse_scorer = make_scorer(mean_squared_error, squared=False)
#     scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=rmse_scorer)

#     return scores.mean()

# # Create a study and optimize the objective function
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)

# Training

In [13]:
%%time
# Wall time: 11min 5s
from sklearn.ensemble import ExtraTreesRegressor

print(datetime.datetime.now(),'*************** Start running Tree ***************')

regressor = ExtraTreesRegressor(n_estimators=100, max_depth=10, random_state=42)  #specifying max_depth=10 reduced r-square significantly from 0.7x to 0.3x
regressor.fit(X_train, y_train)

output_path = './output/ExtraTreesRegressorl_' + str(datetime.datetime.now()) + '.pkl'

import pickle

with open(output_path, 'wb') as file:
    pickle.dump(regressor, file)
print("Model saved as ",output_path) #Wall time: 1min

2024-11-18 16:17:09.267995 *************** Start running Tree ***************
Model saved as  ./output/ExtraTreesRegressorl_2024-11-18 16:18:44.338740.pkl
CPU times: user 1min 34s, sys: 397 ms, total: 1min 34s
Wall time: 1min 35s


In [14]:
regressor.get_params()

{'bootstrap': False,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': 10,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}

# Model evaluation

In [10]:
%%time 
# Load the model back
import pickle

output_path = './output/ExtraTreesRegressorl_2024-11-18 15:16:57.855590.pkl'

with open(output_path, 'rb') as file:
    regressor = pickle.load(file)
print("Model loaded successfully!")

Model loaded successfully!


In [15]:
%%time

import numpy as np
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_error, r2_score

y_pred = regressor.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(datetime.datetime.now(),'Best Model RMSE: %.3f' % rmse)

r2 = r2_score(y_test, y_pred)
print(datetime.datetime.now(),'Best Model R-squared: %.3f' % r2)

2024-11-18 16:18:46.043477 Best Model RMSE: 16.420
2024-11-18 16:18:46.046006 Best Model R-squared: 0.343
CPU times: user 1.57 s, sys: 0 ns, total: 1.57 s
Wall time: 1.56 s


# SHAP

In [ ]:
%%time
#  ~1min
import shap

print(datetime.datetime.now(),'*************** Start running SHAP ***************')
explainer = shap.TreeExplainer(regressor, approximate=True)

2024-11-18 15:59:04.272589 *************** Start running SHAP ***************


In [ ]:
%%time

X_subset = X_test[:100] 
shap_values = explainer(X_subset)

In [ ]:
%%time
print(datetime.datetime.now(),'*************** Start running SHAP importance ***************')
plt.rcParams.update({'font.size': 16})
fig, ax = plt.subplots(1,1)  #,figsize=(6,9)
shap_importance = shap_values.abs.mean(0).values
sorted_idx = shap_importance.argsort()
plt.barh(range(len(sorted_idx)), shap_importance[sorted_idx], align='center',color="#4c72b0") #'turquoise'
plt.yticks(range(len(sorted_idx)), np.array(X.columns)[sorted_idx])
plt.xlabel('SHAP Importance')
# plt.savefig('./output/feature_importance_shap.pdf', bbox_inches='tight')